In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pygam import LinearGAM, s, f
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('assets/output/preprocessedDataset.csv')

In [3]:
# df_temp = df.drop('date', axis=1)
# df_temp['visitor'] = df_temp['visitor'].str.replace(',', '')
# df_temp['visitor'] = df_temp['visitor'].astype(int)

In [4]:
X = df.drop(['date', 'visitor'], axis=1)
y = df['visitor']

In [5]:
# # 데이터 분할
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#
# # 모델 정의
# gam = LinearGAM()
#
# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     'lam': np.arange(0.1, 1.1, 0.1),
#     'n_splines': np.arange(10, 31, 10),
#     'degree': np.arange(1, 4)
# }
#
# # 그리드 서치 객체 생성
# grid_search = GridSearchCV(gam, param_grid=param_grid, cv=5, return_train_score=True)
#
# # 그리드 서치 수행
# grid_search.fit(X_train, y_train)
#
# # 최적 매개변수 조합 및 최적 모델 출력
# best_params = grid_search.best_params_
# print("Best Parameters:", best_params)
# best_model = grid_search.best_estimator_
# print("Best Model:", best_model)
#
# # 테스트 데이터에 대한 예측
# y_pred = best_model.predict(X_test)
#
# # 평가: 평균 제곱 오차 계산
# mse = mean_squared_error(y_test, y_pred)
# print(f"Mean Squared Error: {mse}")

In [6]:
selected_feat = ['sulfur_dioxide_min', 'carbon_monoxide_max', 'ozone_max', 'nitrogen_dioxide_max', 'fine_dust_pm10_max',                 'rainfall_max', 'probability of precipitation_min', 'humidity_min', 'highest temperature_max',                 'lowest temperature_min', 'wind speed_median', 'weekday_1.0' ,'weekday_2.0', 'weekday_3.0','weekday_4.0', 'weekday_5.0', 'weekday_6.0', 'weekday_7.0',  'visitor']
df_selected = df[selected_feat]

X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

In [7]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv('assets/output/preprocessedDataset.csv')

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 상수 열 추가
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# GAM 모델 생성 및 학습
gam = LinearGAM()
gam.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = gam.predict(X_test)

# 평가: 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 평가: 결정 계수(R^2) 계산
r2_gam = r2_score(y_test, y_pred)
print(f"R^2 Score: {r2_gam}")

Mean Squared Error: 81659152.28296266
R^2 Score: 0.2735618808369892


In [8]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Features와 Target 분리
X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

# 모델 학습 및 평가
best_score = float('-inf')
best_k = None

for k in range(2, 11):  # Try different values of k from 2 to 10
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    mse_scores = []
    score_arr = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = LinearGAM()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        mse_scores.append(mse)

        score = model.score(X_test, y_test)
        score_arr.append(score)

    mean_score = np.mean(score_arr)

    if mean_score > best_score:
        best_score = mean_score
        best_k = k

    print(f"K = {k}: Mean Score (R^2) = {mean_score}")

print(f"Best K: {best_k}")
print(f"Best Score: {best_score}")

K = 2: Mean Score (R^2) = 0.2767639462081313
K = 3: Mean Score (R^2) = 0.2790797291775392
K = 4: Mean Score (R^2) = 0.2797201624702008
K = 5: Mean Score (R^2) = 0.2940826127164266
K = 6: Mean Score (R^2) = 0.2894861985898094
K = 7: Mean Score (R^2) = 0.27942148341389056
K = 8: Mean Score (R^2) = 0.2893379078649954
K = 9: Mean Score (R^2) = 0.29257876306378666
K = 10: Mean Score (R^2) = 0.29479380895060625
Best K: 10
Best Score: 0.29479380895060625


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 로드
data = pd.read_csv('assets/output/preprocessedDataset.csv')

# Features와 Target 분리
X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

# 다항 특성 추가
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 평가: 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")

# # 평가: 결정 계수(R^2) 계산
# r2_lin = r2_score(X_test, y_test)
# print(f"R^2 Score: {r2_lin}")

score = model.score(X_test, y_test)
print(f"score: {score}")

Mean Squared Error: 914723536.2566957
score: -8.072517541815914


In [10]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# 데이터 로드
data = pd.read_csv('assets/output/preprocessedDataset.csv')

# 데이터를 k개의 부분집합으로 나누는 k-fold 객체 생성
k = 5  # k-fold 교차 검증에서의 k 값 설정
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Features와 Target 분리
X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

# 다항 특성 추가
poly = PolynomialFeatures(degree=7)
X_poly = poly.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()

# k-fold 교차 검증 수행
mse_scores = []  # 각 fold에서의 평균 제곱 오차(MSE)를 저장할 리스트
score_arr = []

for train_index, test_index in kf.split(X_poly):
    # 훈련 세트, 검증 세트 나누기
    X_train, X_test = X_poly[train_index], X_poly[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 모델 학습
    model.fit(X_train, y_train)

    # 검증 세트 예측
    y_pred = model.predict(X_test)

    # 검증 세트의 평균 제곱 오차 계산
    mse = mean_squared_error(y_test, y_pred)
    score = model.score(X_test, y_test)
    score_arr.append(score)
    mse_scores.append(mse)

# k-fold 교차 검증의 평균 MSE 계산
mean_mse = np.mean(mse_scores)
mean_score = np.mean(score_arr)

print(mean_score)

KeyboardInterrupt: 

In [11]:
def find_best_degree(X, y, min_degree, max_degree, k=5, random_state=42):
    results = []

    for degree in range(min_degree, max_degree+1):
        # 다항 특성 추가
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X)

        # 모델 학습
        model = LinearRegression()

        # k-fold 교차 검증 수행
        kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
        score_arr = []

        for train_index, test_index in kf.split(X_poly):
            # 훈련 세트, 검증 세트 나누기
            X_train, X_test = X_poly[train_index], X_poly[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # 모델 학습
            model.fit(X_train, y_train)

            # 검증 세트의 점수 계산
            score = model.score(X_test, y_test)
            score_arr.append(score)

        # 평균 점수 계산
        mean_score = np.mean(score_arr)

        results.append((degree, mean_score))

    return results

# Features와 Target 분리
X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

min_degree = 1
max_degree = 8

results = find_best_degree(X, y, min_degree, max_degree)

for degree, score in results:
    print(f"Degree: {degree}, Score: {score}")

# Find the best degree and score
best_degree, best_score = max(results, key=lambda x: x[1])

print(f"Best degree: {best_degree}")
print(f"Best score: {best_score}")

KeyboardInterrupt: 

In [15]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

def find_best_degree(X, y, min_degree, max_degree, min_k, max_k, random_state=42):
    results = []
    best_degree = None
    best_score = -np.inf
    best_k = None

    for degree in range(min_degree, max_degree+1):
        # 다항 특성 추가
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X)

        for k in range(min_k, max_k+1):
            # 모델 학습
            model = LinearRegression()

            # k-fold 교차 검증 수행
            kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
            score_arr = []

            for train_index, test_index in kf.split(X_poly):
                # 훈련 세트, 검증 세트 나누기
                X_train, X_test = X_poly[train_index], X_poly[test_index]
                y_train, y_test = y[train_index], y[test_index]

                # 모델 학습
                model.fit(X_train, y_train)

                # 검증 세트의 점수 계산
                score = model.score(X_test, y_test)
                score_arr.append(score)

            # 평균 점수 계산
            mean_score = np.mean(score_arr)
            results.append((degree, k, mean_score))
            print(f"Degree: {degree}, K: {k}, Score: {mean_score}")

            # 현재 degree와 k 값의 점수가 최고 점수보다 높으면 업데이트
            if mean_score > best_score:
                best_degree = degree
                best_score = mean_score
                best_k = k

    # for degree, k, score in results:
    #     print(f"Degree: {degree}, K: {k}, Score: {score}")
    return best_degree, best_score, best_k

X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

min_degree = 2
max_degree = 6
min_k = 2
max_k = 10

best_degree, best_score, best_k = find_best_degree(X, y, min_degree, max_degree, min_k, max_k)

print(f"Best degree: {best_degree}")
print(f"Best score: {best_score}")
print(f"Best k: {best_k}")

Degree: 2, K: 2, Score: 0.25895306962871806
Degree: 2, K: 3, Score: 0.30658694198072184
Degree: 2, K: 4, Score: 0.33032186836426874
Degree: 2, K: 5, Score: 0.3252210633940028
Degree: 2, K: 6, Score: 0.33256864800334296
Degree: 2, K: 7, Score: 0.31723613755822955
Degree: 2, K: 8, Score: 0.3330286829975835
Degree: 2, K: 9, Score: 0.32793413897224705
Degree: 2, K: 10, Score: 0.32056185841589757
Degree: 3, K: 2, Score: -172.9441318264206
Degree: 3, K: 3, Score: -9.126808162075115
Degree: 3, K: 4, Score: -3.8121960824803134
Degree: 3, K: 5, Score: -3.0691732020319558
Degree: 3, K: 6, Score: -2.8415017242851115
Degree: 3, K: 7, Score: -2.462077104034686
Degree: 3, K: 8, Score: -2.349916113651603
Degree: 3, K: 9, Score: -1.9004101319864988
Degree: 3, K: 10, Score: -2.108872865216656
Degree: 4, K: 2, Score: -8.390351365534126
Degree: 4, K: 3, Score: -10.368692273510527
Degree: 4, K: 4, Score: -11.781557623280445
Degree: 4, K: 5, Score: -11.335020483312457
Degree: 4, K: 6, Score: -14.5290255512

KeyboardInterrupt: 